## Advanced GIS: Interactive Web Mapping
#### Final Project | 3/31/2022
**Purpose**: clean and combine housing choice voucher data and neighborhood tabulation geographies for visualization

In [2]:
# Packages and custom functions
import numpy as np
import pandas as pd
import re
import os
import geojson
import geopandas as gpd
import requests as r
from tabula.io import read_pdf
import tabula

def get_county(x):
    c = re.findall('NY New York [\d]{3} (.* County)',x)
    if len(c) > 0:
        return(c[0])
    else:
        return(None)

### Read in voucher data; add fields for filtering, joining, and analysis; filter to just NYC; handle missing data

**Source**: https://www.huduser.gov/portal/datasets/assthsg.html#2009-2021_data, 2021 data

**Documentation**: https://www.huduser.gov/portal/datasets/pictures/dictionary_2021.pdf

**Definition of Missing values**
Some cell entries across variables report no data or are suppressed. In such cases
one of the following codes will apply to such missing values in the downloadable file
"NA" = Not applicable
"-1" = Missing
"-4" = Suppressed (where the cell entry is less than 11 for reported families)
"-5" = Non-reporting (where reporting rates--see % Reported--are less than 50%) 

In [10]:
## Read in voucher data
dat = pd.read_excel("TRACT_MO_WY_2021.xlsx")

## Filter to NY State
dat = dat.loc[dat.states == "NY New York"]

## Check that entire dataset is at census tract level
assert len(dat.loc[dat.sumlevel!=7]) == 0, "Error! Mixed levels of analysis; not all data at census tract level"

## Create county, census tract, and boro fields
dat["county"] = dat["entities"].apply(get_county)
dat["census_tract"] = dat["code"].apply(lambda x: int(x[5:]) if re.match("\d{5}",x) else None)
boros = {"Kings County":3,
        "Queens County":4,
        "Bronx County":2,
        "New York County":1,
        "Richmond County":5}

dat["borocode"] = dat["county"].replace(boros)

## Create aggregate fields for units and occupied for quality checks
dat["est_total_occupied"] = dat["total_units"] * (dat["pct_occupied"] / 100)
dat["diff_occupied_reported"] = dat["est_total_occupied"] - dat["number_reported"]

## Filter to just NYC
cut = dat.loc[dat.county.isin([
    "Kings County",
    "Queens County",
    "Bronx County",
    "Richmond County",
    "New York County"
])]

# Filter to just HCV
hcv = cut.loc[cut.program_label == "Housing Choice Vouchers",
              ["program_label",
               "county",
               "borocode",
               "census_tract",
               "number_reported",
               "people_total",
               "total_units",
               "est_total_occupied",
               "diff_occupied_reported"]].copy()

## Remove suppressed data
hcv = hcv.loc[hcv.number_reported > 0]
hcv.replace(to_replace = -4, value = None, inplace = True)

## Make sure HCV cut is unique on borough and census tract
check = hcv.groupby(["borocode","census_tract"]).aggregate({"program_label":"count"})
assert len(check.loc[check.program_label > 1]) == 0, "Error! Data is not unique on borough and census tract"

## Group HCV by borough and census tract
hcv = hcv.groupby(["program_label","borocode","county","census_tract"]).\
    aggregate({
        "total_units":"max",
        "number_reported":"max",
        "est_total_occupied":"max",
        "diff_occupied_reported":"max",
        "people_total":"max"
    }).\
    reset_index()

### Check data accuracy

In [11]:
## Check that estimated occupied units is not greater than the total number of available units
assert len(hcv.loc[hcv.total_units<hcv.est_total_occupied]) == 0, "Error! Total occupied units greater than total available units"

## Check that the number of reported occupied units is not greater than the total number of available units 
assert len(hcv.loc[hcv.total_units<hcv.number_reported]) == 0, "Error! Reported units greater than total available units"

## Check that only 1 record (Bronx 15800) where reported occupied units is less than estimated occupied units
assert len(hcv.loc[hcv.diff_occupied_reported<0]) == 1,"Error! Total occupied units less than reported units"

## Check that dividing people_total by number_reported is an accurate (within 1 decimal place) estimate of hh size
cut['people_over_reported'] = cut['people_total'] / cut['number_reported']
cut['diff_people_over_reported'] = cut['people_over_reported']-cut['people_per_unit']
assert len(cut.loc[(cut.diff_people_over_reported > 0.1) & (cut.people_total > 0),
        ['people_total','people_over_reported','people_per_unit']]) == 0, "Error! people_total/number_reported not an accurate estimate of hh size"

## Check out the average difference between the number of estimated occupied units vs. number reported
## Note that there is a pretty big spread (up to 40%)
(hcv.diff_occupied_reported / hcv.number_reported*100).describe()


/var/folders/99/zg43g9jx77n46pfkq365c75r0000gn/T/ipykernel_36965/1650092946.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cut['people_over_reported'] = cut['people_total'] / cut['number_reported']
/var/folders/99/zg43g9jx77n46pfkq365c75r0000gn/T/ipykernel_36965/1650092946.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cut['diff_people_over_reported'] = cut['people_over_reported']-cut['people_per_unit']


count    1166.000000
mean       10.831092
std         7.247537
min        -1.085271
25%         6.250000
50%         8.320513
75%        13.142857
max        38.285714
dtype: float64

### # Read in PUMA xwalk, filter to NYC, add borough code, and merge to HVC Dataset

**Data Source:** https://www2.census.gov/geo/docs/maps-data/data/rel/2010_Census_Tract_to_2010_PUMA.txt

**Documentation**: https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2010.html#par_list_0

**Matching County FP to Borough:** https://guides.newman.baruch.cuny.edu/nyc_data

* The Bronx is Bronx County (ANSI / FIPS 36005)
* Brooklyn is Kings County (ANSI / FIPS 36047)
* Manhattan is New York County (ANSI / FIPS 36061)
* Queens is Queens County (ANSI / FIPS 36081)
* Staten Island is Richmond County (ANSI / FIPS 36085)

In [12]:
## Read in
pumas = pd.read_csv("2010_Census_Tract_to_2010_PUMA.csv")

## Filter to NY
nypumas = pumas.loc[(pumas.STATEFP == 36) &
                    (pumas.COUNTYFP.isin([5,47,61,81,85]))].copy()
## Add borocode
boros = {47:3, 81:4, 5:2, 61:1, 85:5}
nypumas["borocode"] = nypumas["COUNTYFP"].replace(boros)

# Merge with HCV dataset and check for uniqueness

## Merge
hcv_puma = hcv.merge(nypumas, 
                     how = 'left',
                     left_on = ['borocode','census_tract'],
                     right_on = ['borocode','TRACTCE'])

## Check that all records merged except those with invalid census tract codes
assert len(hcv_puma.loc[(hcv_puma.census_tract != 999999) &
                        (hcv_puma.TRACTCE.isnull())]) == 0, "Error! Not all records merged" 

## Check that data is unique on borough, tract and puma
check = hcv_puma.groupby(['borocode','census_tract','PUMA5CE']).aggregate({'program_label':'count'}).reset_index()
assert len(check.loc[check.program_label>1]) == 0, "Error! Data is not unique on borough, tract, and puma"

### Scrape PUMA names from map and aggregate HCV data at PUMA level
**Data source:** https://www1.nyc.gov/assets/planning/download/pdf/data-maps/nyc-population/census2010/puma_cd_map.pdf (scraped below)

PUMA <> Community districts and PUMA names

In [13]:
## Scrape pdf data into dataframe
pdf = "https://www1.nyc.gov/assets/planning/download/pdf/data-maps/nyc-population/census2010/puma_cd_map.pdf"
df = read_pdf(pdf, pages = 'all')

## Parse first three columns
table1 = df[0].iloc[:,0:3].dropna(how = "all")
table1.columns = table1.loc[2]
table1 = table1.loc[table1.CD != "CD"]

## Parse second three columns
table2 = df[0].iloc[:,3:5]
table2.columns = table2.loc[2]
table2["CD"] = table2["CD PUMA"].\
    apply(lambda x: re.findall("^(\d+\s*\&*\d*)\s",str(x))[0] if re.match("^(\d+\s*\&*\d*)\s",str(x)) else None)
table2["PUMA"] = table2["CD PUMA"].\
    apply(lambda x: str(x)[-4:] if re.match("\d{4}",str(x)[-4:]) else None)
table2 = table2[["CD","PUMA","PUMA Name"]].dropna(how = "all")

## Add Staten Island rows that get cut off
table3 = pd.DataFrame(
    data = [
        ["1","3903","Port Richmond, Stapleton & Mariner's Harbor"],
        ["2","3902","New Springville & South Beach"],
        ["3","3901","Tottenville, Great Kills & Annadale"]
    ],
    columns = ["CD","PUMA","PUMA Name"])

## Combine tables
table = pd.concat([table1,table2,table3])
table["PUMA"] = table["PUMA"].apply(lambda x: int(x) if x != None else None)

## Correct two PUMA names that get wonky
table["PUMA Name"] = table["PUMA Name"].replace(
    {
        "CD 5Bedford Park, Fordham North & Norwood":"Bedford Park, Fordham North & Norwood",
         "4106Brooklyn Heights & Fort Greene":"Brooklyn Heights & Fort Greene"
    })

## Check that dataset is unique on PUMA
assert (table.PUMA.value_counts() == 1).all() == True, "Error! PUMA codes that map to more than one PUMA"

# Combine HCV dataset and PUMA names + CDs, check for uniqueness, aggregate at PUMA level

## Merge HCV and PUMA names and CDs
hcv_puma_g = hcv_puma.merge(table, how = 'left', left_on = 'PUMA5CE', right_on = 'PUMA')

## Check that data is unique on borough, tract and puma
check = hcv_puma_g.groupby(['borocode','census_tract','PUMA Name']).\
    aggregate({'program_label':'count'}).\
    reset_index()
assert len(check.loc[check.program_label>1]) == 0, "Error! Data is not unique on borough, tract, and puma"

## Aggregate at PUMA level
hcv_puma_g = hcv_puma_g.groupby(['borocode','PUMA','PUMA Name','CD']).\
    aggregate({'est_total_occupied':'sum','number_reported':'sum','people_total':'sum'}).\
    reset_index()

## Add average voucher household size stat
hcv_puma_g['avg_hh_size'] = np.round(hcv_puma_g['people_total']/hcv_puma_g['number_reported'],2)

## Check that all records merged
assert len(hcv_puma.loc[(~hcv_puma.PUMA5CE.isna()) &
                        (~hcv_puma.PUMA5CE.isin(hcv_puma_g.PUMA)),['PUMA5CE']]) == 0, "Error! PUMAS in HCV dataset are missing in PUMA name dataset"

### Sense check aggregated data--does it line up with expectations about vouchers in NYC?

**Data Sources:** https://www1.nyc.gov/site/nycha/section-8/about-section-8.page#:~:text=Approximately%2085%2C000%20Section%208%20vouchers,programs%20in%20New%20York%20City.,
https://www1.nyc.gov/site/hpd/services-and-information/about-section-8.page,
https://www.cbpp.org/research/housing/federal-rental-assistance-fact-sheets#NY

* NYCHA: "Approximately 85,000 Section 8 vouchers ... currently participate in the program."
* HPD: "In total, HPD serives over 39,000 households in all five boroughs."
* CBPP: "Number of Households Receiving Major Types of Federal Rental Assistance in New York. Housing Choice Vouchers: 232,000" for all of New York State

Estimated total: 85K + 39K = **124K housholds**, about half the total for New York State

In [14]:
print("Total reported voucher households: " + "{:,.0f}".format(hcv_puma_g.number_reported.sum()))
print("Total estimated voucher households (% occupied * total units): " + "{:,.0f}".format(hcv_puma_g.est_total_occupied.sum()))
print("Total people in voucher households: " + "{:,.0f}".format(hcv_puma_g.people_total.sum()))
print("Add data note that total occupied may be overestimating")

Total reported voucher households: 116,683
Total estimated voucher households (% occupied * total units): 129,617
Total people in voucher households: 255,140
Add data note that total occupied may be overestimating


In [15]:
# Descriptives for generating styling
hcv_puma_g.number_reported.describe()

count      54.000000
mean     2160.796296
std      2115.885794
min       170.000000
25%       476.750000
50%      1256.000000
75%      3138.500000
max      8957.000000
Name: number_reported, dtype: float64

### Read in geoJSON datasets, reduce precision, merge with HCV data, write to geoJSON file for mapping

**Source and documentation**: https://data.cityofnewyork.us/Housing-Development/2010-Public-Use-Microdata-Areas-PUMAs-/cwiz-gcty

**Data Source Definition:** NYC Open Data Portal GeoJSON file for PUMAs

In [16]:
## Read in and inspect data
puma_json = gpd.read_file("https://data.cityofnewyork.us/api/geospatial/cwiz-gcty?method=export&format=GeoJSON")
puma_json.head()

,puma,shape_area,shape_leng,geometry
0,3701,97928516.6801,53227.1136077,"MULTIPOLYGON (((-73.89641 40.90450, -73.89636 ..."
1,3702,188993662.321,106167.716965,"MULTIPOLYGON (((-73.86943 40.87813, -73.86950 ..."
2,3703,267643637.74,305269.139107,"MULTIPOLYGON (((-73.78833 40.83467, -73.78931 ..."
3,3704,106216907.291,47970.2030563,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
4,3705,122484260.721,68704.1110155,"MULTIPOLYGON (((-73.87046 40.86663, -73.87042 ..."


In [22]:
## Convert PUMA to string type for merging
hcv_puma_g['PUMA'] = hcv_puma_g['PUMA'].apply(lambda x: str(int(x)))

## Merge 
viz = puma_json.merge(hcv_puma_g,
               how = 'inner',
               left_on = 'puma',
               right_on = 'PUMA')

## Check that all records merged
assert len(hcv_puma_g.loc[~(hcv_puma_g.PUMA.isin(viz.PUMA))]) == 0, "Error! Not all records merged to geoJSON"

## rename PUMA Name to proper variable name
viz.rename(columns = {"PUMA Name":"puma_name"}, inplace = True)

## Write to geoJSON file for reduction on https://mapshaper.org/ and then mapping
viz.to_file("hcv_dat.geojson",driver='GeoJSON')

,puma,shape_area,shape_leng,geometry,borocode,PUMA,puma_name,CD,est_total_occupied,number_reported,people_total,avg_hh_size,id
0,3701,97928516.6801,53227.1136077,"MULTIPOLYGON (((-73.89641 40.90450, -73.89636 ...",2,3701,"Riverdale, Fieldston & Kingsbridge",8,2549.80,2331,4164,1.79,3701
1,3702,188993662.321,106167.716965,"MULTIPOLYGON (((-73.86943 40.87813, -73.86950 ...",2,3702,"Wakefield, Williamsbridge & Woodlawn",12,3466.54,3203,7641,2.39,3702
2,3703,267643637.74,305269.139107,"MULTIPOLYGON (((-73.78833 40.83467, -73.78931 ...",2,3703,"Co-op City, Pelham Bay & Schuylerville",10,985.19,930,1939,2.08,3703
3,3704,106216907.291,47970.2030563,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ...",2,3704,"Pelham Parkway, Morris Park & Laconia",11,2345.37,2190,4652,2.12,3704
4,3705,122484260.721,68704.1110155,"MULTIPOLYGON (((-73.87046 40.86663, -73.87042 ...",2,3705,"Belmont, Crotona Park East & East Tremont",3 & 6,10077.15,8957,18868,2.11,3705
5,3706,43886931.112,51826.0734666,"MULTIPOLYGON (((-73.87773 40.88345, -73.87813 ...",2,3706,"Bedford Park, Fordham North & Norwood",7,6735.36,6234,11308,1.81,3706
6,3708,55881108.2631,35002.6915327,"MULTIPOLYGON (((-73.92478 40.84475, -73.92388 ...",2,3708,"Concourse, Highbridge & Mount Eden",4,6028.34,5403,10202,1.89,3708
7,3709,124117686.784,73288.7561407,"MULTIPOLYGON (((-73.83668 40.81759, -73.83696 ...",2,3709,"Castle Hill, Clason Point & Parkchester",9,5673.38,5192,11540,2.22,3709
8,3710,137759806.784,90062.0136606,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ...",2,3710,"Hunts Point, Longwood & Melrose",1 & 2,7028.17,6233,13637,2.19,3710
9,3801,81247268.6263,62589.0551493,"MULTIPOLYGON (((-73.92641 40.87762, -73.92635 ...",1,3801,"Washington Heights, Inwood & Marble Hill",12,5308.84,4874,7835,1.61,3801


### Convert geoJSON datasets for flood maps to correct crs

**Source and documentation**: https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2020s-100-year-Floodplain-/ezfn-5dsb,
        https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2020s-500-year-Floodplain-/ajyu-7sgg

**Data Source Definition:** NYC Open Data Portal

In [13]:
_500yr_floodplain = gpd.read_file("https://data.cityofnewyork.us/api/geospatial/ajyu-7sgg?method=export&format=GeoJSON")
_500yr_floodplain.to_crs("EPSG:4326").to_file("../data/floodplain_500.geojson")

_100yr_floodplain = gpd.read_file("https://data.cityofnewyork.us/api/geospatial/ezfn-5dsb?method=export&format=GeoJSON")
_100yr_floodplain.to_crs("EPSG:4326").to_file("../data/floodplain_100.geojson")

/Users/hgorman/opt/anaconda3/envs/jan22/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
